In [1]:
from flask import Flask, request, jsonify, session, redirect, render_template_string
import mysql.connector
import time

app.secret_key = 'verysecret'

db = mysql.connector.connect(
    host="crossover.proxy.rlwy.net",
    user="root",
    password="BBYoZaaPxAdwlLnJzHzaowpwXZBWMhCG",
    database="railway",
    port=29472
)

cursor = db.cursor(dictionary=True)

# Admin login page
@app.route("/admin", methods=["GET", "POST"])
def admin_login():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]
        cursor.execute("SELECT * FROM admins WHERE username=%s AND password=%s", (username, password))
        admin = cursor.fetchone()
        if admin:
            session["admin"] = admin["username"]
            return redirect("/dashboard")
        else:
            return "Invalid credentials"
    return render_template_string("""
    <form method="post">
        <input name="username" placeholder="Username"><br>
        <input name="password" type="password" placeholder="Password"><br>
        <input type="submit" value="Login">
    </form>
    """)

# Admin dashboard
@app.route("/dashboard")
def admin_dashboard():
    if "admin" not in session:
        return redirect("/admin")
    cursor.execute("SELECT * FROM appeals")
    rows = cursor.fetchall()
    table = "<h3>Appeals Dashboard</h3><table border=1><tr><th>ID</th><th>Student</th><th>Module</th><th>Reason</th><th>Status</th><th>Action</th></tr>"
    for row in rows:
        table += f"<tr><td>{row['id']}</td><td>{row['student_id']}</td><td>{row['module']}</td><td>{row['reason']}</td><td>{row['status']}</td><td><a href='/resolve/{row['id']}'>Resolve</a></td></tr>"
    table += "</table>"
    return table + "<br><a href='/logout'>Logout</a>"

# Mark as resolved
@app.route("/resolve/<int:appeal_id>")
def resolve_appeal(appeal_id):
    if "admin" not in session:
        return redirect("/admin")
    cursor.execute("UPDATE appeals SET status='resolved' WHERE id=%s", (appeal_id,))
    db.commit()
    return redirect("/dashboard")

# Logout admin
@app.route("/logout")
def logout():
    session.pop("admin", None)
    return redirect("/admin")


NameError: name 'app' is not defined